This notebook demonstrates using vector index to sample. There are different ways to determine the uncertainty: 
min_dist_diff: if true, prioritize the sentences that have distances to two centroids have smaller difference
                if false, then prioritize the sentences that have a smaller difference of the max differences to all centroids (max distance- min distance).  
This notebook use  min_dist_diff=True           

# Import 

In [17]:
pwd

'T:\\VINCI_NLPETL\\workspace\\jianlins\\PycharmProjects\\vectorsearch\\notebooks'

In [1]:
import sklearn_crfsuite
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite import CRF
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from spacy.lang.en import English
from spacy import displacy
from pathlib import Path
import joblib
from spacy.tokens import Doc
from typing import List
import random
from statistics import mean, stdev
from loguru import logger
import sys
import spacy
from CRFWrapper import spans_to_bio, convert_docs, word2features, sent2features,compute_metrics_and_averages,  CRFModel
from ALSampler import SamplingSimulator, ModelSamplingSimulator, VBSamplingSimulator


In [2]:
from medspacy_io.reader.brat_reader import BratDocReader, BratDirReader
import medspacy

## Initiate BratDirReader

In [3]:
cleaned_train_dir=r'..\data\n2c2\cleaned_training'
cleaned_test_dir=r'..\data\n2c2\cleaned_test'
Path(cleaned_train_dir).exists(), Path(cleaned_test_dir).exists()

(True, True)

In [4]:
nlp=spacy.load('en_core_web_sm', disable=['ner'])

In [5]:
dir_reader = BratDirReader(nlp=nlp, schema_file=str(Path(cleaned_train_dir, 'annotation.conf')), support_overlap=True)

## Read eHOST annotations | load from pickles

In [6]:
pickle_file= r'..\data\n2c2\spacy_docs.joblib'

In [7]:
if not Path(pickle_file).exists():
    train_docs=dir_reader.read(txt_dir=cleaned_train_dir)
    test_docs=dir_reader.read(txt_dir=cleaned_test_dir)
    print(len(train_docs), len(test_docs))
    joblib.dump((train_docs, test_docs), pickle_file)
else:
    print(f'{pickle_file} already exists, load them directly')
    # before load from pickle, initiate EhostDirReader or EhostDocReader first, because some Doc extension used to store meta data will not be automatically recreated by loading.
    train_docs, test_docs=joblib.load(pickle_file)

..\data\n2c2\spacy_docs.joblib already exists, load them directly


# Define sampling functions

In [8]:
rounds =10
seed= 14

In [9]:
len(train_docs), len(test_docs)

(303, 202)

### Define CRF Wrapper

In [42]:
faiss_index_path= r'..\data\n2c2\faiss_index_st768_o'

In [11]:
## Get all annotation types: 
annos=set()
for d in train_docs:
    for anno in d.spans.keys():
        annos.add(anno)
print(annos)

{'Reason', 'Dosage', 'Strength', 'Duration', 'Form', 'Route', 'Frequency', 'ADE', 'Drug'}


In [12]:
pickle_embedding_file= r'..\data\n2c2\embedding_df.joblib'
crf_model=CRFModel(anno_types=annos)
embedding_df=joblib.load(pickle_embedding_file)
v_simulator=VBSamplingSimulator(train_docs, total_round=10, modelWrapper=crf_model, eval_docs=test_docs, init_seed=seed, faiss_index_path=faiss_index_path, embedding_df=embedding_df, min_dist_diff=True)

2024-05-22 11:41:36.497 | DEBUG    | ALSampler:__init__:120 - Loading index...
2024-05-22 11:41:37.008 | DEBUG    | ALSampler:__init__:122 - done


### Test run

In [14]:
scores=v_simulator.simulate_rounds(boostrap_times=200)

2024-03-21 13:27:23.341 | INFO     | ALSampler:simulate_rounds:44 - simulate round 0.
2024-03-21 13:27:23.343 | DEBUG    | ALSampler:keep_sample:35 - The first round sampling will be random
2024-03-21 13:27:27.283 | DEBUG    | CRFWrapper:fit:251 - Reset and train CRF model...
2024-03-21 13:28:25.236 | DEBUG    | CRFWrapper:fit:259 - Training complete.
2024-03-21 13:28:25.333 | DEBUG    | CRFWrapper:bootstrap_eval:280 - Predicting eval docs...
2024-03-21 13:29:03.255 | DEBUG    | CRFWrapper:bootstrap_eval:284 - Calculate scores from bootstrapping 200 times
2024-03-21 13:33:32.042 | DEBUG    | CRFWrapper:bootstrap_eval:293 - complete
2024-03-21 13:33:32.181 | DEBUG    | ALSampler:simulate_rounds:53 - {'precision': 0.9435403132345506, 'recall': 0.7792371047764345, 'f1': 0.8535492442836575}
2024-03-21 13:33:32.183 | INFO     | ALSampler:simulate_rounds:44 - simulate round 1.
2024-03-21 13:33:32.184 | DEBUG    | ALSampler:sample_next_round:164 - Calculating centroids...
2024-03-21 13:34:23.

In [22]:
def compute_mean_ci(scores):
    ave=np.mean(scores)
    ci=np.percentile(scores, [2.5, 97.5])
    return ave, ci

summary={'precision': [], 'pl':[], 'pu': [], 'recall': [], 'rl':[], 'ru': [], 'f1':[], 'fl':[], 'fu': []}
for s in scores:    
    for k,v in s.items():
        ave, (l, u)=compute_mean_ci(v)
        summary[k].append(ave)
        summary[k[0]+'l'].append(l)
        summary[k[0]+'u'].append(u)


In [27]:
pd.options.display.float_format='{:,.5f}'.format
pd.DataFrame(summary)

,precision,pl,pu,recall,rl,ru,f1,fl,fu
0,0.94354,0.94023,0.94722,0.77924,0.77152,0.78596,0.85355,0.84848,0.85821
1,0.94132,0.93801,0.94428,0.81212,0.80543,0.81904,0.87196,0.86733,0.87677
2,0.94214,0.93895,0.94591,0.82946,0.82384,0.83594,0.88221,0.87862,0.88646
3,0.94164,0.93800,0.94490,0.83861,0.83264,0.84407,0.88714,0.88304,0.89083
4,0.94462,0.94090,0.94780,0.84626,0.84028,0.85227,0.89274,0.88857,0.89689
5,0.94274,0.93997,0.94555,0.85351,0.84840,0.85956,0.89590,0.89251,0.89947
6,0.94453,0.94166,0.94743,0.85460,0.84974,0.85919,0.89732,0.89388,0.90079
7,0.94517,0.94161,0.94854,0.85837,0.85353,0.86351,0.89968,0.89633,0.90308
8,0.94870,0.94503,0.95247,0.85556,0.85149,0.86017,0.89972,0.89693,0.90310


## Avoid reconstruct dataframe  sent_df by save and filtering later. Benchmark..

In [15]:
scores=v_simulator.simulate_rounds(boostrap_times=200)

2024-03-21 21:22:17.252 | INFO     | ALSampler:simulate_rounds:44 - simulate round 0.
2024-03-21 21:22:17.254 | DEBUG    | ALSampler:keep_sample:35 - The first round sampling will be random
2024-03-21 21:22:21.120 | DEBUG    | CRFWrapper:fit:251 - Reset and train CRF model...
2024-03-21 21:23:15.617 | DEBUG    | CRFWrapper:fit:259 - Training complete.
2024-03-21 21:23:15.718 | DEBUG    | CRFWrapper:bootstrap_eval:280 - Predicting eval docs...
2024-03-21 21:23:51.402 | DEBUG    | CRFWrapper:bootstrap_eval:284 - Calculate scores from bootstrapping 200 times
2024-03-21 21:28:08.105 | DEBUG    | CRFWrapper:bootstrap_eval:293 - complete
2024-03-21 21:28:08.247 | DEBUG    | ALSampler:simulate_rounds:53 - {'precision': 0.9435403132345506, 'recall': 0.7792371047764345, 'f1': 0.8535492442836575}
2024-03-21 21:28:08.249 | INFO     | ALSampler:simulate_rounds:44 - simulate round 1.
2024-03-21 21:28:08.250 | DEBUG    | ALSampler:sample_next_round:166 - Calculating centroids...
2024-03-21 21:28:08.

In [16]:
def compute_mean_ci(scores):
    ave=np.mean(scores)
    ci=np.percentile(scores, [2.5, 97.5])
    return ave, ci

summary={'precision': [], 'pl':[], 'pu': [], 'recall': [], 'rl':[], 'ru': [], 'f1':[], 'fl':[], 'fu': []}
for s in scores:    
    for k,v in s.items():
        ave, (l, u)=compute_mean_ci(v)
        summary[k].append(ave)
        summary[k[0]+'l'].append(l)
        summary[k[0]+'u'].append(u)


In [17]:
pd.options.display.float_format='{:,.5f}'.format
pd.DataFrame(summary)

,precision,pl,pu,recall,rl,ru,f1,fl,fu
0,0.94354,0.94023,0.94722,0.77924,0.77152,0.78596,0.85355,0.84848,0.85821
1,0.94132,0.93801,0.94428,0.81212,0.80543,0.81904,0.87196,0.86733,0.87677
2,0.94214,0.93895,0.94591,0.82946,0.82384,0.83594,0.88221,0.87862,0.88646
3,0.94164,0.93800,0.94490,0.83861,0.83264,0.84407,0.88714,0.88304,0.89083
4,0.94462,0.94090,0.94780,0.84626,0.84028,0.85227,0.89274,0.88857,0.89689
5,0.94274,0.93997,0.94555,0.85351,0.84840,0.85956,0.89590,0.89251,0.89947
6,0.94453,0.94166,0.94743,0.85460,0.84974,0.85919,0.89732,0.89388,0.90079
7,0.94517,0.94161,0.94854,0.85837,0.85353,0.86351,0.89968,0.89633,0.90308
8,0.94870,0.94503,0.95247,0.85556,0.85149,0.86017,0.89972,0.89693,0.90310


## Bootstrap 3 runs

In [13]:
logger.remove()
logger.add(sys.stderr, level='INFO')

1

In [14]:
boostrap_runs=3
total_round=10

In [15]:
random.seed(14)
seeds=[random.randint(1,10000000) for  _ in range(boostrap_runs)]
seeds

[1792286, 8843471, 4142887]

In [43]:
all_scores=[]
embedding_df=joblib.load(pickle_embedding_file)
for si, seed  in enumerate(seeds):
    logger.info(f'start run {si}.')
    pickle_embedding_file= r'..\data\n2c2\embedding_df.joblib'
    crf_model=CRFModel(anno_types=annos)
    v_simulator=VBSamplingSimulator(train_docs, total_round=10, modelWrapper=crf_model, eval_docs=test_docs, init_seed=seed, faiss_index_path=faiss_index_path, embedding_df=embedding_df, min_dist_diff=True)
    scores=v_simulator.simulate_rounds(boostrap_times=200)
    all_scores.append(scores) 

2024-05-23 09:25:02.702 | INFO     | __main__:<module>:4 - start run 0.
2024-05-23 09:25:12.653 | INFO     | ALSampler:simulate_rounds:53 - simulate round 0.
2024-05-23 09:25:12.655 | INFO     | ALSampler:keep_sample:43 - current sampled: 30, remaining: 273
2024-05-23 09:31:21.370 | INFO     | ALSampler:simulate_rounds:53 - simulate round 1.
2024-05-23 09:31:21.773 | INFO     | ALSampler:sample_next_round:203 - distance shape: (47530, 10), max to retrieve 47501 sentences
2024-05-23 09:31:24.157 | INFO     | ALSampler:keep_sample:43 - current sampled: 60, remaining: 243
2024-05-23 09:38:19.044 | INFO     | ALSampler:simulate_rounds:53 - simulate round 2.
2024-05-23 09:38:19.541 | INFO     | ALSampler:sample_next_round:203 - distance shape: (47530, 10), max to retrieve 47501 sentences
2024-05-23 09:38:21.767 | INFO     | ALSampler:keep_sample:43 - current sampled: 90, remaining: 213
2024-05-23 09:46:52.248 | INFO     | ALSampler:simulate_rounds:53 - simulate round 3.
2024-05-23 09:46:52.

In [44]:
joblib.dump(all_scores, r'..\data\vb_sampling\ner_vb_scores.joblib')

['..\\data\\vb_sampling\\ner_vb_scores.joblib']

In [ ]:
def compute_mean_ci(scores):
    ave=np.mean(scores)
    ci=np.percentile(scores, [2.5, 97.5])
    return ave, ci

summary={'precision': [], 'recall': [], 'f1':[]}
for s in all_scores[0]:    
    for k,v in s.items():
        summary[k].append(compute_mean_ci(v))


In [ ]:
pd.DataFrame(summary)

In [45]:
len(v_simulator.sampled)

303

In [47]:
len(v_simulator.remaining)

0